# A10 - Distance de Levenshtein

## Introduction

Ce chapitre traite de la distance de Levenshtein et présente quelques implémentations Python de cette mesure. Il existe de nombreux cas d'utilisation de la distance de Levenshtein. La distance de Levenshtein et les idées sous-jacentes sont largement utilisées dans des domaines tels que l'informatique, la linguistique informatique, et même la bioinformatique, la biologie moléculaire et l'analyse de l'ADN. On peut même mesurer la similarité des mélodies ou des rythmes en musique1. La distance de Levenshtein a largement imprégné notre vie quotidienne. Chaque fois que vous utilisez un programme ou une application utilisant une forme de vérification orthographique et de correction des erreurs, les programmeurs ont très probablement utilisé la "distance d'édition" ou, comme on l'appelle aussi, la "distance de Levenshtein".

<center>
    <img src="img/lv1.png" width="50%">
</center>

Vous avez peut-être déjà rencontré un autre cas d'utilisation possible de ce concept : Imaginez que vous utilisez un dictionnaire Python, dans lequel vous utilisez des chaînes de caractères comme clés.

Examinons l'exemple suivant de dictionnaire contenant des noms de villes des États-Unis, souvent mal orthographiés :

In [ ]:
cities = {"Pittsburgh":"Pennsylvania",
          "Tucson":"Arizona",
          "Cincinnati":"Ohio",
          "Albuquerque":"New Mexico",
          "Culpeper":"Virginia",
          "Asheville":"North Carolina",
          "Worcester":"Massachusetts",
          "Manhattan":"New York",
          "Phoenix":"Arizona",
          "Niagara Falls":"New York"} 

Ainsi, essayer d'obtenir les noms d'état correspondants via les accès au dictionnaire suivants soulèvera des exceptions :
```python
cities["Tuscon"] cities["Pittsburg"] cities["Cincinati"] cities["Albequerque"] 
```

Si un lecteur humain regarde ces fautes d'orthographe, il n'aura aucun problème à reconnaître la ville que vous avez en tête. Le dictionnaire Python, quant à lui, est pédant et impardonnable. Il n'accepte une clé que si elle est exactement identique.

La question est de savoir à quel point deux chaînes de caractères sont similaires. Ce dont nous avons besoin, c'est d'une métrique de similarité des chaînes ou d'une mesure de la "distance" des chaînes.

Une métrique de chaîne est une métrique qui mesure la distance entre deux chaînes de texte. L'une des métriques de chaîne les plus connues est la distance de Levenshtein, également connue sous le nom de distance d'édition. Levenshtein calcule le nombre de substitutions et de suppressions nécessaires pour transformer une chaîne en une autre.

La distance d'édition minimale entre deux chaînes de caractères est le nombre minimal d'opérations d'édition nécessaires pour convertir une chaîne en une autre. Les opérations d'édition peuvent consister en des insertions, des suppressions et des substitutions.

Les ensembles les plus simples d'opérations d'édition peuvent être définis comme suit :

- Insertion d'un seul symbole. Cela signifie que nous ajoutons un caractère à une chaîne de caractères s. Exemple : Si nous avons la chaîne s = "Manhatan", nous pouvons insérer le caractère "t" pour obtenir l'orthographe correcte :

```shell
>>> s = "Manhatan"
>>> s = s[:5] + "t" + s[5:]
>>> print(s)
Manhattan
```

- Suppression d'un seul symbole Exemple :

```shell
>>> s = "Mannhattan"
>>> s = s[:2] + s[3:]
>>> s
'Manhattan'
```

- Substitution d'un seul symbole Dans l'exemple suivant, nous devons remplacer la lettre "o" par la lettre "a" pour obtenir l'orthographe correcte :

```shell
>>> s = "Manhatton"
>>> s = s[:7] + "a" + s[8:]
>>> s
'Manhattan'
```

La distance d'édition minimale entre les deux chaînes "Mannhaton" et "Manhattan" correspond à la valeur 3, car nous avons besoin de trois opérations d'édition de base pour transformer la première en la seconde :

```shell
>>> s = "Mannhaton"
>>> s = s[:2] + s[3:]         # deletion
>>> s
'Manhaton'
>>> s = s[:5] + "t" + s[5:]   # insertion
>>> s
'Manhatton'
>>> s = s[:7] + "a" + s[8:]   # substitution
>>> s
'Manhattan'
```

Nous pouvons attribuer un poids ou des coûts à chacune de ces opérations d'édition, par exemple en fixant chacun d'eux à 1. Il est également possible d'argumenter que les substitutions devraient être plus coûteuses que les insertions ou les suppressions, donc parfois les coûts des substitutions sont fixés à 2.

## Définition mathématique de la distance de Levenshtein

La distance de Levenshtein entre deux chaînes de caractères ```a``` et ```b``` est donnée par $lev_{a,b}(len(a), len(b))$ où $lev_{a,b}(i, j)$ est égal à : 

 - max(i, j) si min(i, j)=0
 - sinon :
 $$min(lev_{a,b}(i-1, j) + 1,lev_{a,b}(i, j-1) + 1,lev_{a,b}(i-1, j-1) + 1_{ai≠bj})$$
 
 
où $1_{a_i≠b_j}$ est la fonction indicatrice égale à 0 lorsque $a_i=b_j$ et égale à 1 sinon, et $lev_{a,b}(i, j)$ est la distance entre les i premiers caractères de a et les j premiers caractères de b.

La distance de Levenshtein a les propriétés suivantes :

- Elle est nulle si et seulement si les chaînes de caractères sont égales.
- Elle est au moins égale à la différence des tailles des deux chaînes de caractères.
- Elle est au plus égale à la longueur de la chaîne la plus longue.
- Inégalité des triangles : La distance de Levenshtein entre deux cordes n'est pas supérieure à la somme de leurs distances de Levenshtein par rapport à une troisième corde.

## Fonction Levenshtein récursive en Python

La fonction Python suivante implémente la distance de Levenshtein de manière récursive :

In [ ]:
def LD(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([LD(s[:-1], t)+1,
               LD(s, t[:-1])+1, 
               LD(s[:-1], t[:-1]) + cost])

    return res

print(LD("Python", "Peithen"))

Cette implémentation récursive est très inefficace car elle recalcule la distance de Levenshtein des mêmes sous-chaînes encore et encore. Nous comptons le nombre d'appels dans la version suivante en utilisant une fonction décorateur.

In [ ]:
from collections import Counter

def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        key = str(args) + str(kwargs)
        helper.c[key] += 1
        return func(*args, **kwargs)
    helper.c = Counter()
    helper.calls = 0
    helper.__name__= func.__name__

    return helper

@call_counter
def LD(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([LD(s[:-1], t)+1,
               LD(s, t[:-1])+1, 
               LD(s[:-1], t[:-1]) + cost])

    return res

print(LD("Python", "Peithen"))

print("LD was called " + str(LD.calls) + " times!")
print(LD.c.most_common())

Nous pouvons constater que cette fonction récursive est très inefficace. La distance de Levenshtein de la chaîne s="" et t="P" a été calculée 5336 fois. Dans la version suivante, nous ajoutons un peu de "mémoire" à notre fonction récursive de Levenshtein en ajoutant un mémo dictionnaire :

In [ ]:
%%capture
%%writefile levenshtein.py

def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        return func(*args, **kwargs)
    helper.calls = 0
    helper.__name__= func.__name__

    return helper


memo = {}
@call_counter
def levenshtein(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    cost = 0 if s[-1] == t[-1] else 1
       
    i1 = (s[:-1], t)
    if not i1 in memo:
        memo[i1] = levenshtein(*i1)
    i2 = (s, t[:-1])
    if not i2 in memo:
        memo[i2] = levenshtein(*i2)
    i3 = (s[:-1], t[:-1])
    if not i3 in memo:
        memo[i3] = levenshtein(*i3)
    res = min([memo[i1]+1, memo[i2]+1, memo[i3]+cost])
    
    return res

In [ ]:
print(levenshtein("Python", "Pethno"))

print("The function was called " + str(levenshtein.calls) + " times!")

La version récursive précédente est maintenant efficace, mais elle comporte un défaut de conception. Nous avons pollué le code avec les déclarations pour mettre à jour notre dictionnaire mem. Bien sûr, la conception est bien meilleure, si nous ne polluons pas notre code en ajoutant la logique de sauvegarde des valeurs dans notre fonction Levenshtein. Nous pouvons également "externaliser" ce code dans un décorateur. La version suivante utilise un décorateur "memoize" pour sauvegarder ces valeurs :

In [ ]:
def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        return func(*args, **kwargs)
    helper.calls = 0
    helper.__name__= func.__name__

    return helper

def memoize(func):
    mem = {}
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in mem:
            mem[key] = func(*args, **kwargs)
        return mem[key]
    return memoizer

@call_counter
@memoize    
def levenshtein(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
    
    res = min([levenshtein(s[:-1], t)+1,
               levenshtein(s, t[:-1])+1, 
               levenshtein(s[:-1], t[:-1]) + cost])

    return res

print(levenshtein("Python", "Peithen"))

print("The function was called " + str(levenshtein.calls) + " times!")

Les appels supplémentaires proviennent du fait que nous avons trois appels inconditionnels comme arguments de la fonction "min".

## Calcul itératif de la distance de Levenshtein

<center>
    <img src="img/lv2.png" width="40%">
</center>

Pour calculer la distance de Levenshtein de manière non récursive, nous utilisons une matrice contenant les distances de Levenshtein entre tous les préfixes de la première chaîne et tous les préfixes de la seconde. Nous pouvons calculer dynamiquement les valeurs de cette matrice. La dernière valeur calculée sera la distance entre les deux chaînes complètes. Il s'agit d'un exemple algorithmique de programmation dynamique ascendante.

L'algorithme fonctionne comme suit :
Nous fixons le coût d'une insertion, d'une suppression et d'une substitution à 1. Nous voulons calculer la distance entre deux chaînes s et t avec ```len(s) == m``` et ```len(t) == n```. On utilise une matrice D, qui contient dans la cellule ```(i,j)``` la distance de Levenshtein entre ```s[:i+1]``` et ```t[:j+1]```. Les valeurs de la matrice seront calculées en commençant par le coin supérieur gauche et en terminant par le coin inférieur droit.

Nous commençons par remplir les cas de base, c'est-à-dire la ligne et la colonne avec l'indice 0. Le calcul dans ce cas signifie que nous remplissons la ligne avec l'indice 0 avec les longueurs des sous-chaînes de t et respectivement remplissons la colonne avec l'indice 0 avec les longueurs des sous-chaînes de s.

Les valeurs de tous les autres éléments de la matrice ne dépendent que des valeurs de leur voisin de gauche, du voisin du haut et du voisin du haut gauche.

Le calcul de ```D(i,j)``` pour i et j supérieur à 0 fonctionne comme suit :```D(i,j)``` signifie que nous calculons la distance de Levenshtein des sous-chaînes ```s[0:i-1]``` et ```t[0:j-1]```. Si les derniers caractères de ces sous-chaînes sont égaux, la distance d'édition correspond à la distance des sous-chaînes```s[0:-1]``` et ```t[0:-1]```, qui peut être vide, si s ou t ne comporte qu'un seul caractère, ce qui signifie que nous utiliserons les valeurs de la $0^{iéme}$ colonne ou ligne. Si les derniers caractères de ```s[0:i-1]``` et ```t[0:j-1]``` ne sont pas égaux, la distance d'édition ```D[i,j]``` sera fixée à la somme de ```1 + min(D[i, j-1], D[i-1, j], D[i-1, j-1])-```.

Nous illustrons cela dans le schéma suivant :

In [ ]:
def iterative_levenshtein(s, t):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
    """

    rows = len(s)+1
    cols = len(t)+1
    dist = [[0 for x in range(cols)] for x in range(rows)]

    # source prefixes can be transformed into empty strings 
    # by deletions:
    for i in range(1, rows):
        dist[i][0] = i

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = 1
            dist[row][col] = min(dist[row-1][col] + 1,      # deletion
                                 dist[row][col-1] + 1,      # insertion
                                 dist[row-1][col-1] + cost) # substitution

    for r in range(rows):
        print(dist[r])
    
 
    return dist[row][col]

print(iterative_levenshtein("flaw", "lawn"))

L'image suivante de la matrice de notre calcul précédent contient - coloré en jaune - le chemin optimal à travers la matrice. Nous commençons par une suppression ("f"), nous conservons le "l" (aucun coût ajouté), puis nous conservons le "a" et le "w". La dernière étape est une insertion, ce qui porte les coûts à 2, soit la distance de Levenstein finale.

Pour les besoins d'un autre exemple, utilisons la distance de Levenshtein pour notre exemple initial de ce chapitre. Nous allons donc virtuellement "revenir" à la ville de New York et à son palpitant arrondissement, Manhattan. Nous le comparons avec une faute d'orthographe "Manahaton", qui est la combinaison de diverses fautes d'orthographe courantes.

In [ ]:
print(iterative_levenshtein("Manhattan", "Manahaton"))

<center>
    <img src="img/lv3.png" width="30%">
</center>

Jusqu'à présent, nous avions des coûts fixes pour les insertions, les suppressions et les substitutions, c'est-à-dire que chacun d'entre eux était fixé à 1.

In [ ]:
def iterative_levenshtein(s, t, costs=(1, 1, 1)):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
        
        costs: a tuple or a list with three integers (d, i, s)
               where d defines the costs for a deletion
                     i defines the costs for an insertion and
                     s defines the costs for a substitution
    """

    rows = len(s)+1
    cols = len(t)+1
    deletes, inserts, substitutes = costs
    
    dist = [[0 for x in range(cols)] for x in range(rows)]

    # source prefixes can be transformed into empty strings 
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = row * deletes

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = col * inserts
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = substitutes
            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + cost) # substitution

    for r in range(rows):
        print(dist[r])
    
 
    return dist[row][col]

# default:
print(iterative_levenshtein("abc", "xyz"))
# the costs for substitions are twice as high as inserts and delets:
print(iterative_levenshtein("abc", "xyz", costs=(1, 1, 2)))
# inserts and deletes are twice as high as substitutes
print(iterative_levenshtein("abc", "xyz", costs=(2, 2, 1)))

La situation dans l'appel à iterative_levenshtein avec les coûts par défaut, c'est-à-dire 1 pour les insertions, les suppressions et les substitutions :

<center>
    <img src="img/lv4.png" width="20%">
</center>

Le contenu de la matrice si nous les substitutions sont deux fois plus coûteuses que les insertions et les suppressions, c'est-à-dire l'appel ```iterative_levenshtein("abc", "xyz", costs=(1, 1, 2))``` :

<center>
    <img src="img/lv5.png" width="20%">
</center>

Nous appelons maintenant iterative_levenshtein("abc", "xyz", costs=(2, 2, 1)), ce qui signifie qu'une substitution est deux fois moins étendue qu'une insertion ou une suppression :

<center>
    <img src="img/lv6.png" width="20%">
</center>

Il est également possible d'avoir des poids individuels pour chaque caractère. Au lieu de passer un tuple avec trois valeurs à la fonction, nous allons utiliser un dictionnaire avec des valeurs pour chaque caractère.

In [ ]:
def iterative_levenshtein(s, t, **weight_dict):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
        
        weight_dict: keyword parameters setting the costs for characters,
                     the default value for a character will be 1
    """

    rows = len(s)+1
    cols = len(t)+1
    
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    w = dict( (x, (1, 1, 1)) for x in alphabet + alphabet.upper())
    if weight_dict:
        w.update(weight_dict)
    
    dist = [[0 for x in range(cols)] for x in range(rows)]

    # source prefixes can be transformed into empty strings 
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = dist[row-1][0] + w[s[row-1]][0]

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = dist[0][col-1] + w[t[col-1]][1]
        
    for col in range(1, cols):
        for row in range(1, rows):
            deletes = w[s[row-1]][0]
            inserts = w[t[col-1]][1]
            subs = max( (w[s[row-1]][2], w[t[col-1]][2]))
            if s[row-1] == t[col-1]:
                subs = 0
            else:
                subs = subs

            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + subs) # substitution

    for r in range(rows):
        print(dist[r])
    
 
    return dist[row][col]

# default:
print(iterative_levenshtein("abx", 
                            "xya", 
                            x=(3, 2, 8),
                            y=(4, 5, 4),
                            a=(7, 6, 6)) )
#print(iterative_levenshtein("abc", "xyz", costs=(1,1,substitution_costs)))

Nous démontrons dans le diagramme suivant comment l'algorithme fonctionne avec les caractères pondérés. Les flèches orange montrent le chemin vers la distance d'édition minimale 11 :

<center>
    <img src="img/lv7.png" width="40%">
</center>
